In [1]:
%cd DS-Unit-2-Build-Week

[WinError 2] The system cannot find the file specified: 'DS-Unit-2-Build-Week'
c:\Users\jblac\OneDrive\Documents\Lambda-JOSH-PC\DS-Unit-2-Build-Week


In [2]:
ls

Volume in drive C is SSD
 Volume Serial Number is E288-DA6D

 Directory of c:\Users\jblac\OneDrive\Documents\Lambda-JOSH-PC\DS-Unit-2-Build-Week

04/22/2020  12:26 PM    <DIR>          .
04/22/2020  12:26 PM    <DIR>          ..
04/22/2020  04:49 PM            40,835 build-week-scrap.ipynb
04/22/2020  12:26 PM                 0 scratch.ipynb
04/21/2020  01:30 PM    <DIR>          wine-deep-learning
04/21/2020  02:14 PM        49,784,334 winemag-data_first150k.csv
04/20/2020  02:57 PM        52,908,706 winemag-data-130k-v2.csv
04/20/2020  02:57 PM        79,279,294 winemag-data-130k-v2.json
               5 File(s)    182,013,169 bytes
               3 Dir(s)  159,106,306,048 bytes free


In [27]:
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor


from sklearn.metrics import accuracy_score





for filename in glob('*.csv'):
    print(filename)

winemag-data-130k-v2.csv
winemag-data_first150k.csv


In [4]:
 df30 = pd.read_csv('winemag-data-130k-v2.csv')
 df50 = pd.read_csv('winemag-data_first150k.csv')

In [5]:
df30.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [6]:
df30.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [7]:
df30 = df30.drop('Unnamed: 0', axis=1)

In [8]:
df50.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [9]:
df50 = df50.drop('Unnamed: 0', axis=1)

In [10]:
# I'm curious if top 150k is actually the top 150k, so let's describe the points column on both dataframes
df30['points'].value_counts()

88     17207
87     16933
90     15410
86     12600
89     12226
91     11359
92      9613
85      9530
93      6489
84      6480
94      3758
83      3025
82      1836
95      1535
81       692
96       523
80       397
97       229
98        77
99        33
100       19
Name: points, dtype: int64

In [11]:
# That one contains 88-100, what about top 150k
df50['points'].value_counts()

87     20747
88     17871
90     15973
86     15573
89     12921
85     12411
84     10708
91     10536
92      9241
83      6048
93      6017
82      4041
94      3462
95      1716
81      1502
80       898
96       695
97       365
98       131
99        50
100       24
Name: points, dtype: int64

In [12]:
# Looks pretty similar, maybe we'll just work with the 150k df
# scratch that, going to use the 130k dataframe, since it has columns to prevent duplicate entries

# Renaming for ease
df = df30.copy()

In [13]:
# random forest / coefficients
# permutation importances

In [14]:
df['points'].value_counts(sort=False)

80       397
81       692
82      1836
83      3025
84      6480
85      9530
86     12600
87     16933
88     17207
89     12226
90     15410
91     11359
92      9613
93      6489
94      3758
95      1535
96       523
97       229
98        77
99        33
100       19
Name: points, dtype: int64

In [15]:
df['price'].value_counts(sort=False)

14.0     3215
32.0     1963
64.0      186
8.0       892
96.0       37
         ... 
319.0       1
39.0      924
767.0       1
142.0       8
540.0       1
Name: price, Length: 390, dtype: int64

In [16]:
df['price'].isnull().sum()

8996

In [17]:
#

train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2) 


In [18]:
train.shape, val.shape, test.shape

((83180, 13), (20796, 13), (25995, 13))

In [19]:
def wrangle(X):
    """Wrangle train, and test sets in the same way"""

    # copy dataframe to minimize casualities
    X = X.copy()

    # since points and price are cardinal we can't replace them with 0
    # because that will affect the mean, mode, median so we drop
    X = X[X['points'].notna()]
    X = X[X['price'].notna()]

    # only nans left are categorical, so we don't care about mean, mode, median
    X.fillna('Missing', inplace=True)

    return X

In [20]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [21]:
target = 'price'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
y_train.value_counts(normalize=False)
X_test = test

In [22]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)
# y_pred

In [23]:
# ORDINAL ENCODER + RANDOM FOREST

# transpose before pipeline
# X_train = X_train.transpose()
# X_val = X_val.transpose()

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

# Validation Accuracy 0.2045782635594968

In [28]:
pipeline = make_pipeline(
    ce.OneHotEncoder(), 
    SimpleImputer(strategy='median'), 
    DecisionTreeRegressor(criterion='mae', max_depth=10, max_features=12)
)

# Validation Accuracy 

In [24]:
X_train.shape, y_train.shape

((77434, 12), (77434,))

In [25]:
X_val.shape, y_val.shape

((19396, 12), (19396,))

In [29]:
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))

MemoryError: Unable to allocate 40.3 GiB for an array with shape (73569, 73570) and data type int64